In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import expon
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf

ds = tf.contrib.distributions
k = tf.keras

plt.rcParams.update({
    "lines.color": "white",
    "patch.edgecolor": "white",
    "text.color": "black",
    "axes.facecolor": "white",
    "axes.edgecolor": "lightgray",
    "axes.labelcolor": "white",
    "xtick.color": "white",
    "ytick.color": "white",
    "grid.color": "lightgray",
    "figure.facecolor": "#222222",
    "figure.edgecolor": "#222222",
    "savefig.facecolor": "#22222200",
    "savefig.edgecolor": "#222222",
    "font.size": 20})


In [ ]:

import tensorflow as tf
ds = tf.contrib.distributions

r_dist = tf.placeholder_with_default(2., shape=())

bkg = ds.MultivariateNormalFullCovariance(loc=[r_dist, 0.],
        covariance_matrix=[[5.,0.],[0.,9.]], name="p0")

sig = ds.MultivariateNormalDiag(loc=[0., 0.],
        scale_diag=[1., 1.], name="p1")

mu = tf.placeholder(shape=(), dtype=tf.float32, name="mu")
mix = ds.Mixture(cat=ds.Categorical(probs=[1.-mu, mu]),
               components = [bkg, sig])


In [ ]:
data_sample = mix.sample(1020, seed=17)
sig_sample = sig.sample(400, seed=17)
bkg_sample = bkg.sample(400, seed=17)

with tf.Session() as sess:
    data_arr = sess.run(data_sample, feed_dict={mu : 20./1000.})
    sig_arr = sess.run(sig_sample)
    bkg_arr = sess.run(bkg_sample)
    

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

ax.scatter(data_arr[:,0],data_arr[:,1], color="black", alpha=0.5, s=20)
ax.set_xlabel("variable 1")
ax.set_ylabel("variable 2")



ax.set_ylim([-10,10])
ax.set_xlim([-10,10])

#fig.savefig("../../iml_workshop_talk/gfx/obs_data_example.svg",
#            bbox_inches='tight')

fig;

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

ax.scatter(bkg_arr[:,0],bkg_arr[:,1], color="blue", alpha=0.5, s=20, label="bkg simulation")
ax.scatter(sig_arr[:,0],sig_arr[:,1], color="red", alpha=0.5,s=20, label="sig simulation")

ax.set_xlabel("variable 1")
ax.set_ylabel("variable 2")



ax.set_ylim([-10,10])
ax.set_xlim([-10,10])

ax.legend(loc=2)

#fig.savefig("../../iml_workshop_talk/gfx/sim_data_example.svg",
#            bbox_inches='tight')

fig;

In [ ]:
x_scan  = np.linspace(-5.,5.,1001, endpoint=True, dtype=np.float32)
y_scan  = np.linspace(-5.,5.,1001, endpoint=True, dtype=np.float32)

X_grid = np.stack(np.meshgrid(x_scan, y_scan),axis=-1)
grid_shape = list(X_grid.shape[:-1])
X_flat = np.reshape(X_grid, [-1,2])

In [ ]:
X_grid_t = tf.placeholder(shape=(None, None, 2), dtype=tf.float32)

mix_grid = mix.log_prob(X_grid_t)
bkg_grid = bkg.log_prob(X_grid_t)
sig_grid = sig.log_prob(X_grid_t)

with tf.Session() as sess:
    mix_grid_arr = sess.run(mix_grid, feed_dict={X_grid_t : X_grid, mu : 0.2})
    bkg_grid_arr = sess.run(bkg_grid, feed_dict={X_grid_t : X_grid})
    sig_grid_arr =  sess.run(sig_grid, feed_dict={X_grid_t : X_grid})
    

lr_mix_bkg = mix_grid_arr - bkg_grid_arr
lr_sig_bkg = sig_grid_arr - bkg_grid_arr

In [ ]:
n_samples = 125000
X_sample_tensors = {}
y_values = {}
p0_sample = "p0_sample"
X_sample_tensors[p0_sample] = bkg.sample(n_samples // 2, seed=7, name="sig_sample") 
y_values[p0_sample] = 0.
p1_sample = "p1_sample"
X_sample_tensors[p1_sample] = sig.sample(n_samples // 2, seed=17, name="bkg_sample") 
y_values[p1_sample] = 1.

samples = {}

with tf.Session() as sess:
    for name, sample_tensor in X_sample_tensors.items():
        samples[name] = {}
        samples[name]["X"] = sess.run(sample_tensor)
        samples[name]["y"] =  y_values[name]*np.ones(sample_tensor.shape[0],
                                                     dtype=np.float32)
train_samples = {}
valid_samples = {}

for name,sample in samples.items():
    keys = sample.keys()
    split_sample = train_test_split(*list(sample.values()),
                                    test_size = 0.4,
                                    random_state=17)
    train_samples[name] = { k : split_sample[i] for k,i in zip(keys,[0,2])}
    valid_samples[name] = { k : split_sample[i] for k,i in zip(keys,[1,3])}
    

X_train = np.concatenate([s["X"] for s in train_samples.values()] )
y_train = np.concatenate([s["y"] for s in train_samples.values()] )
X_valid = np.concatenate([s["X"] for s in valid_samples.values()] )
y_valid = np.concatenate([s["y"] for s in valid_samples.values()] )

i_train = np.arange(y_train.shape[0])
np.random.shuffle(i_train)

X_train = X_train[i_train]
y_train = y_train[i_train]

In [ ]:
model = k.Sequential()
activation = "relu" 
initializer = "glorot_normal" 
model.add(k.layers.Dense(10, activation=activation,
            kernel_initializer=initializer,
        name="dense_0", input_shape=(2,)))
model.add(k.layers.Dense(10, activation=activation,
          kernel_initializer=initializer,
          name="dense_1"))
model.add(k.layers.Dense(1, activation="sigmoid", name="output"))

In [ ]:
optim = k.optimizers.SGD(lr=0.01)
model.compile(optim,"binary_crossentropy")

In [ ]:
model = k.models.load_model("keras_bce_SGD_lr_0p01_100_epochs.h5")

In [ ]:
history = model.fit(X_train, y_train, batch_size=128, epochs=50,validation_data=(X_valid, y_valid))

In [ ]:
X_flat = np.reshape(X_grid, [-1,2])
y_flat = model.predict(X_flat, batch_size=1024)
y_grid = np.reshape(y_flat, grid_shape)

lr_nn_approx =  np.log(y_grid)-np.log(1.-y_grid)

In [ ]:
plt.tight_layout()
fig, axs = plt.subplots(1,3,figsize=(30,8))

titles = [r"Analytical $\ln(p(x|H_1)/p(x|H_0))$",
          r"Analytical $\ln(p_s(x)/p_b(x))$",
          r"DNN approximation $\ln(p_s(x)/p_b(x))$"]
levels = [None,
          np.linspace(-21,3,9,endpoint=True),
          np.linspace(-21,3,9,endpoint=True)]

for i,lr in enumerate([lr_mix_bkg, lr_sig_bkg, lr_nn_approx]):
    con = axs[i].contourf(x_scan, y_scan, lr, levels=levels[i])
    fig.colorbar(con, ax=axs[i],)
    axs[i].set_xlabel("variable 1")
    axs[i].set_ylabel("variable 2")
    axs[i].set_title(titles[i],color='white')
    axs[i].set_ylim([-5,5])
    axs[i].set_xlim([-5,5])
    
#fig.savefig("../../iml_workshop_talk/gfx/clf_lr_approximation.svg",transparent=True,bbox_inches='tight')
fig;

In [ ]:
bkg_pred = model.predict_proba(valid_samples["p0_sample"]["X"])
sig_pred = model.predict(valid_samples["p1_sample"]["X"])

dat_pred = model.predict_proba(data_arr)


In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

bins = np.linspace(0., 1., 11, endpoint=True)
bkg_hist,_ = np.histogram(bkg_pred,bins=bins,density=True)
sig_hist,_ = np.histogram(sig_pred,bins=bins,density=True)
dat_hist,_ = np.histogram(dat_pred,bins=bins)

width = bins[:-1] - bins[1:]
center = (bins[:-1] + bins[1:]) / 2


#ax.bar(center, dat_hist, align='center', width=width)
bkg_normed = 1000.*bkg_hist/bkg_hist.sum()
sig_normed = 20.*sig_hist/sig_hist.sum()
bkg_bar = ax.bar(center, bkg_normed, align='center', color="blue", alpha=0.5, width=width, label="background")
sig_bar = ax.bar(center, sig_normed, align='center', color="red", bottom=bkg_normed, alpha=0.5, width=width, label="signal")
dat_bar = ax.errorbar(center, dat_hist, yerr=np.sqrt(dat_hist),fmt='k.', capsize=4, elinewidth=2, label="data")

ax.legend()
ax.set_xlabel("NN classifier output per event")
ax.set_ylabel("counts")
ax.set_xlim([0.,1.])
ax.set_facecolor("white")

#fig.savefig("../../iml_workshop_talk/gfx/clf_sig_bkg_data.svg",bbox_inches='tight',edgecolor='none')

In [ ]:
shift = np.array([[1.,0.]])
bkg_pred_up = model.predict_proba(valid_samples["p0_sample"]["X"]+shift)
bkg_pred_dw = model.predict_proba(valid_samples["p0_sample"]["X"]-shift)

bins = np.linspace(0., 1., 11, endpoint=True)
bkg_hist,_ = np.histogram(bkg_pred,bins=bins,density=True)
bkg_hist_up,_ = np.histogram(bkg_pred_up,bins=bins,density=True)
bkg_hist_dw,_ = np.histogram(bkg_pred_dw,bins=bins,density=True)
sig_hist,_ = np.histogram(sig_pred,bins=bins,density=True)


In [ ]:
import neyman.inferences as ni
import neyman.models as nm


def int_quad_lin(alpha, c_nom, c_up, c_dw):
    "Three-point interpolation, quadratic inside and linear outside"
    
    alpha_t = tf.tile(tf.expand_dims(alpha,axis=-1),[1, tf.shape(c_nom)[0]])
    a = 0.5*(c_up+c_dw)-c_nom
    b = 0.5*(c_up-c_dw)
    ones = tf.ones_like(alpha_t)
    switch = tf.where(alpha_t < 0.,
                      ones*tf.expand_dims(c_dw-c_nom, axis=0),
                      ones*tf.expand_dims(c_up-c_nom, axis=0))
    abs_var = tf.where(tf.abs(alpha_t) > 1., 
                      (2*b+tf.sign(alpha_t)*a)*(alpha_t-tf.sign(alpha_t))+switch,
                      a*tf.pow(alpha_t,2)+b*alpha_t)
    return c_nom+abs_var

def poisson(x, rate):
    "float64  poisson pdf (avoid numerical inacurracies)"
    x_d = tf.cast(x, tf.float64)
    rate_d = tf.cast(rate, tf.float64)
    log_rate_d = tf.log(rate_d)
    p_d = x_d*log_rate_d - tf.lgamma(tf.convert_to_tensor(1.,dtype=tf.float64)+x_d)-rate_d
    return tf.cast( p_d, tf.float32)

# expected shapes for background (nom/up/down variation for interpolation)
c_bkg_nom_ph = tf.placeholder(dtype=tf.float32, shape=(None,), name="c_bkg_nom_ph") 
c_bkg_dw_ph = tf.placeholder(dtype=tf.float32, shape=(None,), name="c_bkg_dw_ph") 
c_bkg_up_ph = tf.placeholder(dtype=tf.float32, shape=(None,), name="c_bkg_up_ph") 
# expected shapes for signal (no parameters)
c_sig_ph = tf.placeholder(dtype=tf.float32, shape=(None,), name="c_sig_ph")

# expected number of signal and background events
n_bkg_ph = tf.placeholder_with_default(10000., shape=(), name="n_bkg_ph")
n_sig_ph = tf.placeholder_with_default(100., shape=(), name="n_sig_ph")

# model parameters (input specified by placeholders)
mu_ph = tf.placeholder(dtype=tf.float32, shape=(None,), name="mu_ph")
r_dist_ph = tf.placeholder(dtype=tf.float32, shape=(None,), name="r_dist_ph")

# auxiliary measurement parameters
r_dist_scale_ph = tf.placeholder_with_default(0.2, shape=(), name="theta_scale_ph")

# distribution of nuissance parameters
r_dist_rv = nm.Normal(loc=tf.ones_like(r_dist_ph)*2.0,
                      scale=tf.ones_like(r_dist_ph)*r_dist_scale_ph,
                      value=r_dist_ph,
                      name="r_dist_dist")

# background shape as a function of r_dist
c_bkg = int_quad_lin((r_dist_rv-2.0)/r_dist_scale_ph, c_bkg_nom_ph,
                     c_bkg_up_ph, c_bkg_dw_ph)

# expected events ([batch, bin])
mu = tf.expand_dims(mu_ph,-1, name="mu_expanded")
expected = mu*n_sig_ph*c_sig_ph+n_bkg_ph*c_bkg

# placeholder for data/asimov
observed_ph = tf.placeholder(dtype=tf.float32, shape=(None,), name="observed_ph")

# likelihood
poisson_pdf = poisson(observed_ph, expected)
nll = -tf.reduce_sum(poisson_pdf ,-1)
r_dist_ext = -r_dist_rv.log_prob(r_dist_rv)
nll_ext = nll+r_dist_ext 

# hessians and likelihoods
h_nll, g_nll =  ni.batch_hessian(nll, pars=[mu_ph, r_dist_rv])
h_nll_ext, g_nll_ext = ni.batch_hessian(nll_ext, pars=[mu_ph, r_dist_rv])

# covariance without constraints (only POI)
c_nll_poi = tf.matrix_inverse(h_nll[:,:1,:1], name="c_nll_poi")
c_nll = tf.matrix_inverse(h_nll, name="c_nll")
c_nll_ext = tf.matrix_inverse(h_nll_ext, name="c_nll_ext")
# profile grads and hess (only nuissance par)
g_nll_prof = g_nll_ext[:,1:]
h_nll_prof = h_nll_ext[:,1:,1:]
c_nll_prof = tf.matrix_inverse(h_nll_prof, name="c_nll_prof")

# newton step
newton_step =  tf.matmul(c_nll_prof, g_nll_prof[:,:, tf.newaxis])[:,0,0]

In [ ]:
shape_phs = {c_bkg_nom_ph : bkg_hist/bkg_hist.sum(),
             c_bkg_dw_ph : bkg_hist_dw/bkg_hist_dw.sum(),
             c_bkg_up_ph : bkg_hist_up/bkg_hist_up.sum(),
             c_sig_ph : sig_hist/sig_hist.sum()}

norm_phs = {n_bkg_ph : 1000.,
            n_sig_ph : 40.}

par_phs = {mu_ph : [1.]}


In [ ]:
par_phs = {mu_ph : [1.],
           r_dist_rv: [2.]}

feed_dict = {**shape_phs, **norm_phs, **par_phs}

minima = {}
with tf.Session() as sess:
    asimov_data = sess.run(expected, feed_dict=feed_dict)
    print("asimov data: ", asimov_data[0])
    asimov_phs = {observed_ph : asimov_data[0]}
    feed_dict = {**feed_dict, **asimov_phs}
    minima  = sess.run({ t : t for t in [c_nll_ext, c_nll_poi, g_nll_ext ] }, feed_dict=feed_dict)

print("error mu no nuis:", np.sqrt(np.diag(minima[c_nll_poi][0])))
print("error mu and nuis:", np.sqrt(np.diag(minima[c_nll_ext][0])))

In [ ]:

mu_scan = np.linspace(0.02,1.98, 101, endpoint=True, dtype=np.float32)
par_phs = {mu_ph : mu_scan,
           r_dist_ph: np.ones_like(mu_scan)*2.}

feed_dict = {**shape_phs, **norm_phs, **par_phs, **asimov_phs}

no_nuis = {}
profiled = {}
with tf.Session() as sess:
    no_nuis[nll_ext] = sess.run(nll_ext, feed_dict=feed_dict)
    feed_dict[r_dist_ph] = feed_dict[r_dist_ph]
    for i in range(10):
        newton_step_arr = sess.run(newton_step , feed_dict=feed_dict)
        feed_dict[r_dist_ph] =  feed_dict[r_dist_ph]-newton_step_arr
    profiled[nll_ext] = sess.run(nll_ext, feed_dict=feed_dict)

direct_lr_no_nuis = no_nuis[nll_ext]
direct_lr_nuis = profiled[nll_ext]

In [ ]:
direct_lr_no_nuis

In [ ]:
fig, axs = plt.subplots(2, figsize=(14,16))


ax = axs[0]
width = bins[:-1] - bins[1:]
center = (bins[:-1] + bins[1:]) / 2

ax.bar(center, sig_hist, align='center', width=width, label="signal",
       fill=None,linewidth=2.5, linestyle='-',edgecolor='red' )

ax.bar(center, bkg_hist_up, align='center',width=width, label="bkg shift up",
       fill=None, linewidth=2.5, linestyle='--',edgecolor='darkorange')
ax.bar(center, bkg_hist, align='center', color="blue", alpha=0.5, width=width, label="bkg nom",
       fill=None,linewidth=2.5, linestyle='-',edgecolor='blue')
ax.bar(center, bkg_hist_dw, align='center', width=width, label="bkg shift dw",
       fill=None,linewidth=2.5, linestyle='--',edgecolor='green')

ax.legend()
ax.set_xlim([0.,1.])


ax.set_xlabel("NN classifier output per event")
ax.set_ylabel("density")


ax = axs[1]

ax.plot(mu_scan, no_nuis[nll_ext]-no_nuis[nll_ext].min(),
        color="green",label="without nuissance pars")
ax.plot(mu_scan, profiled[nll_ext]-profiled[nll_ext].min(),
        color="red",label="with nuissance pars (profiled)")

ax.hlines(y=0.5, xmin=0., xmax=2., linestyles='dashed')
ax.set_xlabel("parameter of interest $\mu$")
ax.set_ylabel("$\Delta(\mathcal{-\ln L})$")

ax.set_xlim([0.3,1.7])
ax.set_ylim([-0.05,1.25])


ax.legend()


#fig.savefig("../../iml_workshop_talk/gfx/clf_systematic_effect.svg",bbox_inches='tight',edgecolor='none')


fig;

In [ ]:

mu_scan = np.linspace(0.3,1.6, 101, endpoint=True, dtype=np.float32)
r_dist_scan = np.linspace(1.95,2.05, 101, endpoint=True, dtype=np.float32)
par_phs = {mu_ph : [1],
           theta_ph: [0.0],
           r_dist_ph: r_dist_scan}

feed_dict = {**shape_phs, **norm_phs, **par_phs, **asimov_phs}

nll_surface = np.empty([mu_scan.shape[0],r_dist_scan.shape[0]], dtype=np.float32)


with tf.Session() as sess:
    for i, mu_val in enumerate(mu_scan):
        feed_dict[mu_ph] = [mu_val]
        nll_surface[i] = sess.run(nll_ext, feed_dict=feed_dict)


In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

#im = ax.imshow(nll_surface)
pcm = ax.contourf(r_dist_scan,mu_scan,nll_surface-nll_surface.min(), levels=list(np.linspace(0.,6.,13,endpoint=True)))
fig.colorbar(pcm, ax=ax)
ax.set_title("$\Delta(\mathcal{-\ln L})$ surface", color="white")

ax.set_xlabel("nuisance parameter (bkg mean)")
ax.set_ylabel("parameter of interest $\mu$")

#fig.savefig("../../iml_workshop_talk/gfx/likelihood_ratio_learning.svg",bbox_inches='tight',edgecolor='none')

fig;

In [ ]:
r_dist_scan = np.linspace(1.8,2.2, 101, endpoint=True, dtype=np.float32)
par_phs = {mu_ph : [1.0],
           theta_ph: [0.0],
           r_dist_ph : r_dist_scan}

feed_dict = {**shape_phs, **norm_phs, **par_phs, **asimov_phs}

arrs = {}
with tf.Session() as sess:
    feed_dict = {**feed_dict, **asimov_phs}
    arrs[r_dist_ext]= sess.run(r_dist_ext, feed_dict=feed_dict)
    arrs[nll_ext] = sess.run(nll_ext, feed_dict=feed_dict)

fig, ax = plt.subplots(figsize=(10,8))

ax.plot(r_dist_scan, arrs[r_dist_ext]-arrs[r_dist_ext].min(), color="blue")
ax.plot(r_dist_scan, arrs[nll_ext]-arrs[nll_ext].min(),color="red")

ax.set_ylim([-0.1, 1.0])
fig;

In [ ]:
model = k.models.load_model("shift_0.2_n_bins_2_bs_512_lr_0.0001.h5")

In [ ]:
x_scan  = np.linspace(-5.,5.,1001, endpoint=True, dtype=np.float32)
y_scan  = np.linspace(-5.,5.,1001, endpoint=True, dtype=np.float32)

X_grid = np.stack(np.meshgrid(x_scan, y_scan),axis=-1)
grid_shape = list(X_grid.shape[:-1])
X_flat = np.reshape(X_grid, [-1,2])

X_flat = np.reshape(X_grid, [-1,2])
y_flat = model.predict(X_flat, batch_size=1024)
y_grid = np.reshape(y_flat, grid_shape+[2])

In [ ]:
bkg_pred = model.predict_proba(valid_samples["p0_sample"]["X"])[:,1]
sig_pred = model.predict(valid_samples["p1_sample"]["X"])[:,1]

dat_pred = model.predict_proba(data_arr)[:,1]


In [ ]:
bkg_pred

In [ ]:
sig_hist

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

bins = np.linspace(0., 0.4, 5, endpoint=True)
bkg_hist,_ = np.histogram(bkg_pred,bins=bins,density=True)
sig_hist,_ = np.histogram(sig_pred,bins=bins,density=True)
dat_hist,_ = np.histogram(dat_pred,bins=bins)

width = bins[:-1] - bins[1:]
center = (bins[:-1] + bins[1:]) / 2


#ax.bar(center, dat_hist, align='center', width=width)
bkg_normed = 1000.*bkg_hist/bkg_hist.sum()
sig_normed = 20.*sig_hist/sig_hist.sum()
bkg_bar = ax.bar(center, bkg_normed, align='center', color="blue", alpha=0.5, width=width, label="background")
sig_bar = ax.bar(center, sig_normed, align='center', color="red", bottom=bkg_normed, alpha=0.5, width=width, label="signal")
dat_bar = ax.errorbar(center, dat_hist, yerr=np.sqrt(dat_hist),fmt='k.', capsize=4, elinewidth=2, label="data")

ax.legend()
ax.set_xlabel("NN classifier output per event")
ax.set_ylabel("counts")
ax.set_xlim([0.,0.4])
ax.set_facecolor("white")

#fig.savefig("../../iml_workshop_talk/gfx/clf_sig_bkg_data.svg",bbox_inches='tight',edgecolor='none')

In [ ]:
shift = np.array([[1.,0.]])
bkg_pred_up = model.predict_proba(valid_samples["p0_sample"]["X"]+shift)
bkg_pred_dw = model.predict_proba(valid_samples["p0_sample"]["X"]-shift)

bins = np.linspace(0., 0.4, 5, endpoint=True)
bkg_hist,_ = np.histogram(bkg_pred,bins=bins,density=True)
bkg_hist_up,_ = np.histogram(bkg_pred_up,bins=bins,density=True)
bkg_hist_dw,_ = np.histogram(bkg_pred_dw,bins=bins,density=True)
sig_hist,_ = np.histogram(sig_pred,bins=bins,density=True)


In [ ]:
shape_phs = {c_bkg_nom_ph : bkg_hist/bkg_hist.sum(),
             c_bkg_dw_ph : bkg_hist_dw/bkg_hist_dw.sum(),
             c_bkg_up_ph : bkg_hist_up/bkg_hist_up.sum(),
             c_sig_ph : sig_hist/sig_hist.sum()}

norm_phs = {n_bkg_ph : 1000.,
            n_sig_ph : 40.}

par_phs = {mu_ph : [1.]}


In [ ]:
par_phs = {mu_ph : [1.],
           r_dist_rv: [2.]}

feed_dict = {**shape_phs, **norm_phs, **par_phs}

minima = {}
with tf.Session() as sess:
    asimov_data = sess.run(expected, feed_dict=feed_dict)
    print("asimov data: ", asimov_data[0])
    asimov_phs = {observed_ph : asimov_data[0]}
    feed_dict = {**feed_dict, **asimov_phs}
    minima  = sess.run({ t : t for t in [c_nll_ext, c_nll_poi, g_nll_ext ] }, feed_dict=feed_dict)

print("error mu no nuis:", np.sqrt(np.diag(minima[c_nll_poi][0])))
print("error mu and nuis:", np.sqrt(np.diag(minima[c_nll_ext][0])))

In [ ]:
mu_scan = np.linspace(0.02,1.98, 101, endpoint=True, dtype=np.float32)
par_phs = {mu_ph : mu_scan,
           r_dist_ph: np.ones_like(mu_scan)*2.}

feed_dict = {**shape_phs, **norm_phs, **par_phs, **asimov_phs}

no_nuis = {}
profiled = {}
with tf.Session() as sess:
    no_nuis[nll_ext] = sess.run(nll_ext, feed_dict=feed_dict)
    feed_dict[r_dist_ph] = feed_dict[r_dist_ph]
    for i in range(10):
        newton_step_arr = sess.run(newton_step , feed_dict=feed_dict)
        feed_dict[r_dist_ph] =  feed_dict[r_dist_ph]-newton_step_arr
    profiled[nll_ext] = sess.run(nll_ext, feed_dict=feed_dict)

new_lr_no_nuis = no_nuis[nll_ext]
new_lr_nuis = profiled[nll_ext]

In [ ]:
plt.tight_layout()
fig, ax = plt.subplots(figsize=(10,8))

title= r"log decision surface (n_bins=2)"
level= np.linspace(-5.,0.,11,endpoint=True)


ax.plot(mu_scan,direct_lr_no_nuis-direct_lr_no_nuis.min(),"--",
        color="green",label="classification: without nuissance pars")
ax.plot(mu_scan, direct_lr_nuis-direct_lr_nuis.min(),"--",
        color="red",label="classification: with nuissance pars")
ax.plot(mu_scan,new_lr_no_nuis-new_lr_no_nuis.min(),
        color="green",label="direct: without nuissance pars")
ax.plot(mu_scan, new_lr_nuis-new_lr_nuis.min(),
        color="red",label="direct: with nuissance pars")

ax.hlines(y=0.5, xmin=0., xmax=2., linestyles='dashed')

ax.set_xlabel("parameter of interest $\mu$")
ax.set_title("Profile Likelihood", color="white")
ax.set_ylabel("$\Delta(\mathcal{-\ln L})$")

ax.set_xlim([0.2,1.8])
ax.set_ylim([-0.05,1.25])

ax.legend()
fig.savefig("../../iml_workshop_talk/gfx/profile_likelihood_comparison.svg",bbox_inches='tight',edgecolor='none')


fig;